In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json

In [2]:
all_letters = string.printable
n_letters = len(all_letters)
def letterToIndex(letter):
    return all_letters.find(letter)
def linesToTensor(lines):
    line_length = 15000
    if max([ len(line) for line in lines]) < line_length:
        line_length = max( [len(line) for line in lines] )
    #line_length = max( [len(line) for line in lines] )
    #xx = [max(len(line)-15000,0) for line in lines]
    #print float(np.sum(xx)) / float(np.sum([len(line) for line in lines]))
    tensor = torch.zeros(len(lines), line_length, n_letters)
    for b, line in enumerate(lines): 
        line = line[:15000]
        for li, letter in enumerate(line):
            tensor[b][li + line_length - len(line)][letterToIndex(letter)] = 1

    return tensor

In [3]:
class LangModel(nn.Module):
    def __init__(self, preTrained='True', input=100):
        super(LangModel, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, 128, 3, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(128, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.cuda()
        h0 = ( Variable(torch.zeros(3, text.size(0), 128)).cuda(),  Variable(torch.zeros(3, text.size(0), 128)).cuda())

        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred


In [4]:
"""
chat_data='./exp_chat/{}.npy'.format(game_id)
chat_arr=np.load(chat_data)
window_text=[]
for i in range(len(chat_arr)):
    temp=''
    for idx in range(7):
        if i+idx<len(chat_arr):
            temp+=chat_arr[i+idx]
    window_text.append(temp)
"""

NameError: name 'game_id' is not defined

In [54]:
text=linesToTensor(window_text)

In [58]:
text.size()

torch.Size([2056, 2700, 100])

In [11]:
import torch.utils.data as data

class chat_ds(data.Dataset):
    def __init__(self,game_id):
        self.window_text=[]
        #####game id 전체로 바꿀것##############
        chat_data='./exp_chat/{}.npy'.format(game_id)
        chat_arr=np.load(chat_data)
        #window_text=np.zeros([len(text)])
        for i in range(len(chat_arr)):
            temp=''
            for idx in range(7):
                if i+idx<len(chat_arr):
                    temp+=chat_arr[i+idx]
            self.window_text.append(temp)
        self.text=linesToTensor(self.window_text)

        with open('../tsvt/ocr/hi_time.json','r') as f:
            labels=json.load(f)

        label=labels[game_id]
        
        self.label_gt=torch.zeros((len(self.text),))
        for (x,y) in label:
            start=int(x.split(':')[0])*60+int(x.split(':')[1])
            end=int(y.split(':')[0])*60+int(y.split(':')[1])
            for i in range(start,end+1):
                self.label_gt[i]=1
        self.label_gt=self.label_gt.long()
    def __len__(self):
        return len(self.label_gt)
    def __getitem__(self,index):
        return self.text[index],self.label_gt[index]



In [12]:
game_id='102844212428895431'
a=chat_ds(game_id)

In [13]:
loader=torch.utils.data.DataLoader(a,batch_size=100)

In [14]:
###### model load #####
model=LangModel().cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [93]:
#train set
for iter in range(10):
    model.train()
    for inputs,labels in (loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer = torch.optim.SGD(model.parameters(), 0.1)
        optimizer.zero_grad()
        output=model(inputs)

        loss=criterion(output,labels)
        loss.backward()
        optimizer.step()

    #validation
    model.eval()
    pred_sum = 0#model output
    gt_sum = 0#label
    tp_sum=0
    fp_sum=0
    fn_sum=0
    with torch.no_grad():
        for it, (inputs,labels) in enumerate(loader):
            count+=1
            inputs = inputs.cuda()
            labels = labels.cuda()
            output=model(inputs)

            TP,FP,TN,FN,pred_len, gt_len=fmeasure(output.cpu(),labels.cpu())
            tp_sum += TP
            fp_sum += FP
            fn_sum += FN
            pred_sum += pred_len
            gt_sum += gt_len

        precision = tp_sum/(tp_sum+fp_sum)
        recall = tp_sum / (tp_sum+fn_sum)
        f1 = (2*precision*recall / (precision + recall)) * 100
        print( tp_sum, fp_sum, fn_sum)
        print('[{}/{}], prec:{}, recall:{}, f1:{}'.format(it, len(loader), precision, recall, f1))

923 1133 0
[20/21], prec:0.4489299610894942, recall:1.0, f1:61.967103054716354
923 1133 0
[20/21], prec:0.4489299610894942, recall:1.0, f1:61.967103054716354


KeyboardInterrupt: 

In [88]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len